# CHAPTER 5
## Deep Q-Network

In [1]:
import gym
import random
import time

import atari_wrappers as atari

In [ ]:
def make_env(env_name, fire=True, frames_num=2, noop_num=30, skip_frames=True):
    
    env = gym.make(env_name)
    
    if skip_frames:
        env = atari.MaxAndSkipEnv(env) ## Return only every skip-th frame
        
    if fire:
        env = atari.FireResetEnv(env) ## Fire at the beggining
        
    env = atari.NoopResetEnv(env,noop_max=noop_num)
    env = atari.WarpFrame(env) ## Reshape image
    env = atari.FrameStack(env, frames_num) ## Stack last 4 frames
    
    return env